In [18]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch_geometric
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

In [6]:
# Import data
data = pd.read_pickle('../data/connectivity_compliance_matrices.pkl')
data.rename(columns={'ρ':'rho'}, inplace=True)

# Calculate mean of E1, E2, E3
data['mean_E'] = data[['E1', 'E2', 'E3']].mean(axis=1)

# Extract relevant features
X = data[['connectivity_matrix', 'compliance_matrix', 'rho', 'mean_E']]
display(X.head())

,connectivity_matrix,compliance_matrix,rho,mean_E
0,"[[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...","[[16.2442166588418, -4.390589663678543, -4.392...",0.3,0.061528
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[22.194443522542244, -7.184492662134119, -7.1...",0.3,0.045108
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[21.44865815819801, -6.467827160497426, -6.44...",0.3,0.046636
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[15.247634909853652, -3.931170890685161, -3.9...",0.3,0.065567
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[23.16273271145176, -5.511611078765939, -5.66...",0.3,0.043226


In [9]:
# Construct sample graph data consisting of edge index and node features (rho)
sample = X.iloc[0]
conn_mat = sample['connectivity_matrix']
C_mat = sample['compliance_matrix']
edge_index = np.array(np.nonzero(conn_mat)).T
rho = sample['rho']
display(sample['connectivity_matrix'])

rhos = np.array([rho] * conn_mat.shape[0])
num_neighbors = np.sum(conn_mat, axis=1)
node_features = np.column_stack((rhos, num_neighbors))
print(f"Node features:\n{node_features}")

sample_data = Data(edge_index=torch.tensor(edge_index, dtype=torch.long), y=torch.tensor(C_mat, dtype=torch.float32))
print(f"Edge index:\n{sample_data.edge_index}")
print(f"Compliance matrix:\n{sample_data.y}")

array([[0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.]])

Node features:
[[0.3 2. ]
 [0.3 2. ]
 [0.3 2. ]
 [0.3 2. ]
 [0.3 4. ]
 [0.3 0. ]
 [0.3 2. ]
 [0.3 2. ]
 [0.3 0. ]
 [0.3 0. ]
 [0.3 2. ]]
Edge index:
tensor([[ 0,  1],
        [ 0,  4],
        [ 1,  0],
        [ 1,  7],
        [ 2,  3],
        [ 2, 10],
        [ 3,  2],
        [ 3,  4],
        [ 4,  0],
        [ 4,  3],
        [ 4,  6],
        [ 4, 10],
        [ 6,  4],
        [ 6,  6],
        [ 7,  1],
        [ 7,  7],
        [10,  2],
        [10,  4]])
Compliance matrix:
tensor([[16.2442, -4.3906, -4.3921,  0.0000,  0.0000,  0.0000],
        [-4.3906, 16.2539, -4.3922,  0.0000,  0.0000,  0.0000],
        [-4.3921, -4.3922, 16.2602,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, 38.9590,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, 39.0762,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 39.0137]])


In [17]:
def construct_graph_data(row):
    # Extract matrices and features
    conn_mat = row['connectivity_matrix']
    C_mat = row['compliance_matrix']
    rho = row['rho']
    mean_E = row['mean_E']

    # Normalize compliance matrix
    min_val = np.min(C_mat)
    max_val = np.max(C_mat)
    C_mat = (C_mat - min_val) / (max_val - min_val)

    # Create edge index from connectivity matrix
    edge_index = np.array(np.nonzero(conn_mat)).T
    num_nodes = conn_mat.shape[0]

    # Create node features: rho and number of neighbors
    rhos = np.array([rho] * num_nodes)
    num_neighbors = np.sum(conn_mat, axis=1)

    # Create node features
    x = torch.tensor(np.column_stack((rhos, num_neighbors)), dtype=torch.float)
    graph_data = Data(x=x,
                      edge_index=torch.tensor(edge_index, dtype=torch.long).t().contiguous(),
                      y=torch.tensor(C_mat.flatten(), dtype=torch.float32),
                      mean_E=torch.tensor([mean_E], dtype=torch.float32))
    
    return graph_data

# Apply the function to the entire DataFrame to create a list of graph data objects
graph_data_list = X.apply(construct_graph_data, axis=1).tolist()
print(f"Constructed {len(graph_data_list)} graph data objects.")
print(f"Sample graph data object:\n{graph_data_list[0]}")
print(f"Sample normalized compliance matrix:\n{graph_data_list[0].y.reshape(6, 6)}")

Constructed 2624 graph data objects.
Sample graph data object:
Data(x=[11, 2], edge_index=[2, 18], y=[36], mean_E=[1])
Sample normalized compliance matrix:
tensor([[4.7474e-01, 3.7080e-05, 1.8203e-06, 1.0104e-01, 1.0104e-01, 1.0104e-01],
        [3.7080e-05, 4.7497e-01, 0.0000e+00, 1.0104e-01, 1.0104e-01, 1.0104e-01],
        [1.8203e-06, 0.0000e+00, 4.7511e-01, 1.0104e-01, 1.0104e-01, 1.0104e-01],
        [1.0104e-01, 1.0104e-01, 1.0104e-01, 9.9730e-01, 1.0104e-01, 1.0104e-01],
        [1.0104e-01, 1.0104e-01, 1.0104e-01, 1.0104e-01, 1.0000e+00, 1.0104e-01],
        [1.0104e-01, 1.0104e-01, 1.0104e-01, 1.0104e-01, 1.0104e-01, 9.9856e-01]])


In [19]:
train_data, test_data = train_test_split(graph_data_list, test_size=0.2, random_state=42)
print(f"Training set size: {len(train_data)}")
print(f"Testing set size: {len(test_data)}")

Training set size: 2099
Testing set size: 525


In [ ]:
class GNN_v1(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN_v1, self).__init__()
        self.conv = torch_geometric.nn.GCN